In [171]:
#hide
#skip
%config Completer.use_jedi = False
%config IPCompleter.greedy=True
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [172]:
# default_exp fastrl.fastai.loop

In [173]:
# export
# Python native modules
import os,sys,json
from copy import deepcopy,copy
from typing import *
import types
import logging
import inspect
from itertools import chain,product
from functools import partial
# Third party libs
from fastcore.all import *
import numpy as np
# Local modules
from fastrl.core import test_in

IN_IPYTHON=False

_logger=logging.getLogger(__name__)

In [174]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbverbose.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display=Display(visible=0,size=(400,300))
    display.start()

# Loop
> fastrl concept of generic loop objects. 

The goal for Loops is to make it easy to customize, and know how sections of code connects
to other parts.

Ideally, a loop can consist of inner loops to have a format:
```python
FitLoop
BatchLoop
ClassicTrain
BatchLoop
StepLoop
ValidStepLoop
```
Which might have a structure:
```json
{
"FitLoop": [
    "FitLoop (epoch)": [
        "ClassicTrain (train)": [
            "BatchLoop (batches)": [
                "StepLoop (pred)": [],
                "StepLoop (loss)": [],
                "StepLoop (backward)": [],
                "StepLoop (step)": [],
                "StepLoop (zero_grad)": []
            ]
        ],
        "ClassicTrain (valid)": [
            "BatchLoop (batches)": [
                "ValidStepLoop (pred)": [],
                "ValidStepLoop (loss)": [],
            ]
        ]
    ]
}
```

In [175]:
# export
EVENT_ORDER_MAPPING={}
PREFIXES=['before_','on_','after_','failed_','finally_']

def isevent(o): return issubclass(o.__class__,Event)
class EventException(Exception):pass
def _default_raise(_placeholder): raise 

class KwargSetAttr(object):
    def __setattr__(self,name,value):
        "Allow setting attrs via kwarg."
        super().__setattr__(name,value)

class Events(KwargSetAttr,L):
    def __init__(self,items=None,postfix=None,prefix=None,item_iter_hint='prefix',
                 order=0,parent_event=None,*args,**kwargs):
        store_attr(but='items')
        super().__init__(items=items,*args,**kwargs)
        
    def flat(self):
        return Events(chain.from_iterable(self),
                      postfix=self.postfix,prefix=self.prefix,
                      item_iter_hint=self.item_iter_hint,order=self.order,
                      parent_event=self.parent_event)

    def __lt__(self,o:'Event'): return self.order<o.order    
    def todict(self): 
        return {getattr(o,self.item_iter_hint):o for o in self}
    
    def __repr__(self): 
        if len(self)==0: return super().__repr__()
        return '\n'.join([str(o) for o in self])
    def run(self):
        for o in self: o.run()                                                  # fastrl.skip_traceback

class Event(KwargSetAttr):
    def __init__(self,
                 function:Callable,
                 loop=None,
                 override_name=None,
                 override_qualname=None,
                 override_module=None,
                 order=None
                ):
        store_attr()
        if self.function==noop and self.prefix=='failed_':
            self.function=_default_raise
        # We set the order over the entire Loop definition
        if self.order is None:
            if self.outer_name not in EVENT_ORDER_MAPPING: self.order=1
            else: self.order=EVENT_ORDER_MAPPING[self.outer_name]
            EVENT_ORDER_MAPPING[self.outer_name]=self.order+1

        # self.original_name=self.function.__module__+'.'+self.function.__qualname__
            
        if self.name.startswith('_') or not any(self.name.startswith(pre) for pre in PREFIXES):
            raise EventException(f'{self.name} needs to start with any {PREFIXES}')
            
        self.cbs=L()
        
    def climb(self):
        "Returns a generator that moves up to the parent/root event"
        yield self
        if self.loop is not None:
            yield from self.loop.climb()
            
    @property
    def level(self):
        level=1
        for e in self.climb():
            level+=1
        return level
        
    @classmethod
    def from_override_name(cls,name,**kwargs):
        return cls(noop,override_name=name,**kwargs)
        
    def set_cbs(self,cbs=None):
        if cbs is not None:
            self.cbs=L((cb() if isinstance(cb, type) else cb) for cb in L(cbs) 
                       if hasattr(cb,self.name) and (not cb.call_on or any([isrelevent(cb,e) for e in self.climb()])))
            
    @property
    def root_loop(self): return self.loop.root_loop
    def __call__(self,*args,**kwargs): 
        ret=self.function(self.loop,*args,**kwargs)                             # fastrl.skip_traceback
        for cb in self.cbs: cb_ret=getattr(cb,self.name)()
        return ret

    def __lt__(self,o:'Event'): return self.order<o.order
    @property
    def name(self): return ifnone(self.override_name,self.function.__name__)
    @property
    def module(self): return ifnone(self.override_module,self.function.__module__)
    @property
    def qualname(self): return ifnone(self.override_qualname,self.function.__qualname__)
    @property
    def prefix(self): return self.name.split('_')[0]+'_'
    @property
    def postfix(self): return '_'.join(self.name.split('_')[1:])
    @property
    def outer_name(self): return self.module+'.'+self.qualname.split('.')[0]
    @property
    def original_name(self): 
        return self.function.__module__+'.'+self.function.__qualname__
    
    def __repr__(self): return self.module+'.'+self.name
    def with_inner(self):
        return (self,Events(postfix=self.postfix,
                            prefix=self.prefix+'inner',
                            order=self.order))

event=Event

In [176]:
# export
class Loops(L):
    def run(self):
        for o in self: o.run()                                                  # fastrl.skip_traceback

class Loop(object):
    verbose=False
    def __init__(self):
        self.parent_loop=None
        self.parent_event=None
        
        events=Events(inspect.getmembers(self)).map(Self[-1]).filter(isevent).sorted()
        events.map(Event.__setattr__,name='loop',value=self)
        # 1. Make Events have the same module as the function being run
        # 2. Convert the Events to Events+Inner Events
        # 3. Convert [(Event,[]*inner events*)...] to [Event,[]*inner events*...]
        # 4. Sure they are sorted correctly
        self.default_events=Events(PREFIXES)\
            .map(Event.from_override_name,override_module=events[0].module)\
            .map(Event.with_inner)\
            .flat()\
            .sorted()                                                           
        self.events=events.sorted().map(Event.with_inner).flat().sorted()
        self.sections=groupby(self.events,Self.postfix())
        for k,v in self.sections.items():
            self.sections[k]=merge(self.default_events.map(copy).todict(),
                                   Events(v).todict())
        
    def copy(self): return self.__class__()

    def run(self):
        try:                                                                    # fastrl.skip_traceback
            for v in self.sections.values(): run_section(v)                     # fastrl.skip_traceback
        except Exception as e:
            e._show_loop_errors=self.verbose
            raise
    
    def __repr__(self): return str(self.sections) #json.dumps(self.sections,default=str)

In [177]:
class Outer(Loop):
    @event
    def before_step(self) :  print('before_step')
    @event
    def on_step(self)     :  print('on_step')
    @event
    def after_step(self)  :  print('after_step')
    @event
    def failed_step(self) :  print('failed_step')
    @event
    def finally_step(self):  print('finally_step')
 
    @event
    def before_jump(self) :  print('before_jump')
    @event
    def on_jump(self)     :  print('on_jump')
    @event
    def after_jump(self)  :  print('after_jump')
    @event
    def failed_jump(self) :  print('failed_jump')
    @event
    def finally_jump(self):  print('finally_jump')

class Inner(Loop):
    call_on=L(Outer.on_step,Outer.after_step,Outer.finally_jump)
    
    @event
    def before_iteration(self) : print('before_iteration')
    @event
    def on_iteration(self)     : print('on_iteration')
    @event
    def after_iteration(self)  : print('after_iteration')
    @event
    def failed_iteration(self) : print('failed_iteration')
    @event
    def finally_iteration(self): print('finally_iteration')

class FailingInner(Loop):
    call_on=L(Inner.finally_iteration)
    
    @event
    def on_force_fail(self):                    
        print('on_force_fail')
        raise Exception

In [178]:
# export
def run_section(section:Dict):
    try:
        section['before_']()
        section['before_inner'].run()                                           # fastrl.skip_traceback
        section['on_']()
        section['on_inner'].run()
        section['after_']()
        section['after_inner'].run()                                            # fastrl.skip_traceback
    except Exception as ex:
        try:     
            section['failed_']()                                                # fastrl.skip_traceback
            raise
        finally: 
            section['failed_inner'].run()                                       # fastrl.skip_traceback
    finally:
        section['finally_']()
        section['finally_inner'].run()                                          # fastrl.skip_traceback

In [179]:
# export
def eq_loops(a:Loop,b:Loop): return a.__class__==b.__class__

@with_cast
def connect_loops2loop(loops:Loops,to_loop):
    loops=loops.map(Self.copy())
    to_events=to_loop.events.filter(isevent).map(Self.original_name()) 
    for from_loop in loops.filter(eq_loops,b=to_loop,negate=True):
        for call_on in from_loop.call_on:
            if call_on.original_name in to_events:
                _from_loop=from_loop.copy()
                
                _from_loop.parent_event=to_loop.sections[call_on.postfix][call_on.prefix]
                _from_loop.parent_loop=to_loop
                
                to_loop.sections[call_on.postfix][call_on.prefix+'inner'].extend([_from_loop])
                connect_loops2loop(loops,_from_loop)
    return to_loop

In [180]:
connect_loops2loop(Loops(FailingInner(),Inner()),Outer()).sections

{'step': {'failed_': __main__.failed_step,
  'failed_inner': [],
  'before_': __main__.before_step,
  'before_inner': [],
  'on_': __main__.on_step,
  'on_inner': {'iteration': {'failed_': __main__.failed_iteration, 'failed_inner': [], 'before_': __main__.before_iteration, 'before_inner': [], 'on_': __main__.on_iteration, 'on_inner': [], 'after_': __main__.after_iteration, 'after_inner': [], 'finally_': __main__.finally_iteration, 'finally_inner': {'force_fail': {'failed_': __main__.failed_, 'failed_inner': [], 'before_': __main__.before_, 'before_inner': [], 'on_': __main__.on_force_fail, 'on_inner': [], 'after_': __main__.after_, 'after_inner': [], 'finally_': __main__.finally_, 'finally_inner': []}}}},
  'after_': __main__.after_step,
  'after_inner': {'iteration': {'failed_': __main__.failed_iteration, 'failed_inner': [], 'before_': __main__.before_iteration, 'before_inner': [], 'on_': __main__.on_iteration, 'on_inner': [], 'after_': __main__.after_iteration, 'after_inner': [], 'fi

In [181]:
connect_loops2loop(Loops(FailingInner(),Inner()),Outer()).sections['step']['on_inner'][0].sections

{'iteration': {'failed_': __main__.failed_iteration,
  'failed_inner': [],
  'before_': __main__.before_iteration,
  'before_inner': [],
  'on_': __main__.on_iteration,
  'on_inner': [],
  'after_': __main__.after_iteration,
  'after_inner': [],
  'finally_': __main__.finally_iteration,
  'finally_inner': {'force_fail': {'failed_': __main__.failed_, 'failed_inner': [], 'before_': __main__.before_, 'before_inner': [], 'on_': __main__.on_force_fail, 'on_inner': [], 'after_': __main__.after_, 'after_inner': [], 'finally_': __main__.finally_, 'finally_inner': []}}}}

In [182]:
connect_loops2loop(FailingInner(),Inner()).sections

{'iteration': {'failed_': __main__.failed_iteration,
  'failed_inner': [],
  'before_': __main__.before_iteration,
  'before_inner': [],
  'on_': __main__.on_iteration,
  'on_inner': [],
  'after_': __main__.after_iteration,
  'after_inner': [],
  'finally_': __main__.finally_iteration,
  'finally_inner': {'force_fail': {'failed_': __main__.failed_, 'failed_inner': [], 'before_': __main__.before_, 'before_inner': [], 'on_': __main__.on_force_fail, 'on_inner': [], 'after_': __main__.after_, 'after_inner': [], 'finally_': __main__.finally_, 'finally_inner': []}}}}

In [183]:
# export
def _skip_traceback(s):
    return in_('# fastrl.skip_traceback',s)
    
def ipy_handle_exception(self, etype, value, tb, tb_offset):
    ## Do something fancy
    stb = self.InteractiveTB.structured_traceback(etype,value,tb,tb_offset=tb_offset)
    if not getattr(value,'_show_loop_errors',True):
        tmp,idxs=[],L(stb).argwhere(_skip_traceback)
        prev_skipped_idx=idxs[0] if idxs else 0
        for i,s in enumerate(stb):
            if i in idxs and i-1!=prev_skipped_idx: 
                msg='Skipped Loop Code due to # fastrl.skip_traceback found in source code,'
                msg+=' please use Loop(...verbose=True) to view loop tracebacks\n'
                tmp.append(msg)
            if i not in idxs:
                tmp.append(s)
            else:
                prev_skipped_idx=i
        stb=tmp
    ## Do something fancy
    self._showtraceback(type, value, stb)

if IN_IPYTHON:
    get_ipython().set_custom_exc((Exception,),ipy_handle_exception)
    

In [184]:
if False:
    Outer.verbose=False
    Outer().run(L(Inner(),FailingInner()),OuterCallback())

In [185]:
if False:
    Outer.verbose=True
    Outer().run(L(Inner(),FailingInner()),OuterCallback())

In [186]:
connect_loops2loop(Loops(FailingInner(),Inner()),Outer()).run()

before_step
on_step
before_iteration
on_iteration
after_iteration
finally_iteration
on_force_fail
failed_step
finally_step


type: 

In [188]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbverbose.cli import *
    make_readme()
    notebook2script()
    notebook2html()

converting /home/fastrl_user/fastrl/nbs/index.ipynb to README.md
Converted 00_core.ipynb.
Converted 00_nbdev_extension.ipynb.
Converted 02_fastai.exception_test.ipynb.
Converted 02_fastai.loop.ipynb.
No export destination, ignored:
# export
# Python native modules
import os,sys
from copy import deepcopy,copy
from typing import *
import types
import logging
import inspect
from itertools import chain,product
from functools import partial
# Third party libs
from fastcore.all import *
import numpy as np
# Local modules
from fastrl.core import test_in

IN_IPYTHON=False

_logger=logging.getLogger(__name__)
No export destination, ignored:
# export
class _Prop(object):
        
    def __getattr__(self,k): 
        self._attr=k
        return self
    def __call__(self,o): return getattr(o,self._attr)


class _PropCls(object):
    __metaclass__ = _Prop
        
    def __getattr__(self,k): return getattr(_Prop(),k)
    def __call__(self): return self

Prop=_PropCls()
No export destination, ign

type: 'NoneType' object has no attribute 'start'